## Import packages needed for all of these API Calls

In [1]:
import requests
import math
import pprint
import json

# Creating a function to list all jobs for a given dbt Cloud environment

### Creating the function

In [2]:
def get_dbt_cloud_list_job_ids_per_env(api_key, account_id, environment_id):
    
    # define headers
    headers = {
        "Authorization": f"Token {api_key}",
        "Content-Type": "application/json",
    }
    
    # set base url for dbt Cloud
    base_url = f"https://cloud.getdbt.com/api/v2/accounts/{account_id}/jobs"

    # set the url that will get called
    query_params = {
     'environment_id': environment_id,
     'limit': 100,
     'offset': 0
    }
    
    # get the response
    response = requests.get(base_url, headers=headers, params=query_params).json()

    # get the total number of jobs for the env
    total_jobs = response["extra"]["pagination"]["total_count"]
    call_offset = response["extra"]["filters"]["offset"]
    
    # parse response json
    response_json = response["data"]

    # creating a list for all jobs
    all_jobs = []

    # append data to the list
    all_jobs.append(response_json)

    # we make sure that we get all jobs is there are greater than 100 jobs for the given env
    if len(response_json) > 0 and total_jobs > 100:

        while True:

            # Increment offset by limit for the next page
            call_offset += 100

            # Break the loop if we have collected all items
            if call_offset >= total_jobs:
                break

            # set it as a query param
            query_params['offset'] = call_offset

            # make the api call
            response = requests.get(base_url, headers=headers, params=query_params).json()
            call_offset = response["extra"]["filters"]["offset"]
            response_json = response["data"]

            # append the additonal data to the list
            all_jobs.append(response_json)

    # logging that pagination isn't needed if 100 jobs or less
    elif len(response_json) > 0 and total_jobs <= 100:

        # logging that pagination is not needed
        print("No need to do pagination, less than 101 jobs")
        
    else:
        # raise error that the job were not found
        raise ValueError(f"There was an error getting job ids or there are no jobs created for the environment_id {environment_id}")

    # flatten the jobs list 
    flattened_jobs_list = [item for sublist in all_jobs for item in sublist]

    # simplify the data returned
    simplified_jobs = [
                            {'job_name': job['name'], 'job_id': job['id'], 'environment_id': job['environment_id']}
                            for job in flattened_jobs_list
                      ]

    # return list of jobs
    return simplified_jobs

### Using the function

In [3]:
# Example usage:
api_key = '<< dbt cloud token goes here >>'
account_id = 12345  # Replace with your account ID
environment_id = 678910

# use the function
job_ids_for_env = get_dbt_cloud_list_job_ids_per_env(api_key, account_id, environment_id)

# print them out for example
pprint.pp(job_ids_for_env)

No need to do pagination, less than 101 jobs
[{'job_name': 'Weekly Refresh Job', 'job_id': 106102, 'environment_id': 105436},
 {'job_name': 'Airport Data Refresh',
  'job_id': 494432,
  'environment_id': 105436},
 {'job_name': 'Customer Reporting Job',
  'job_id': 494574,
  'environment_id': 105436},
 {'job_name': 'Parameterized Customer Job',
  'job_id': 533070,
  'environment_id': 105436},
 {'job_name': 'Build a Semantic Layer Export ',
  'job_id': 548096,
  'environment_id': 105436}]


# END OF SCRIPT